In [53]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
import json
from scipy import stats
import os.path

In [54]:
def nclusters(data, threshold):
    data = [float(el) for el in data]
    data = sorted(data)
    start = data[0]
    max_val = start + threshold
    c = (start, max_val)
    cluster = dict()
    for i in data:
        if i <= max_val:
            if c in cluster.keys():
                cluster[c] += 1
            else:
                cluster[c] = 1
        else:
            max_val = i + threshold
            c = (i, max_val)
            cluster[c] = 1
    #ora ho il dizionario con i cluster di una run
    C_num = len(data)**2
    C_den = 0
    for k in cluster.keys():
        C_den += cluster[k]*cluster[k]
    C = C_num / C_den
    return C

In [55]:
def entropy(opinions, n, nbins):
    bincounts, bin_edges = np.histogram(opinions, bins = np.linspace(0, 1, nbins))
    probabilities = bincounts/n
    entr = stats.entropy(probabilities)
    return entr

def nodeperc_withininterval(opinions, center, width):
    sortedops = sorted(opinions)
    sortedopsarr = np.array(sortedops)
    withinrange = np.where(np.logical_and(sortedopsarr>=(center-width), sortedopsarr<=(center+width)))
    withinrange = withinrange[0]
    perc = len(withinrange) / len(sortedops)
    return perc

def average_opinions(opinions):
    a = np.array(opinions)
    return np.average(a)

def std_opinions(opinions):
    a = np.array(opinions)
    return np.std(a)

def median_opinions(opinions):
    a = np.array(opinions)
    return np.median(a)

maxentr = entropy(np.random.uniform(0.0, 1.0, 250), 250, 11)


In [83]:
model = "rewiring"
gr = 0.0
finalfile = f"{model} results final new.csv"

In [84]:
#create one aggregate file
import os
aggrfile = open("finals/{}".format(finalfile), "w+")
header = "model,graph,n,p,pr,eps,gam,max_it\n"
aggrfile.write(header)
aggrfile.close()
aggrfile = open("finals/{}".format(finalfile), "a+")
for graph in ['er', 'ba']:
    if graph == "er":
        p = 0.1
    else:
        p=5
    for pr in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
        for eps in [0.2, 0.3, 0.4]:
            for gam in [0.0, 0.4, 0.8, 1.2, 1.6]:
                name = f"{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000"
                s = f"{model},{graph},250,{p},{pr},{eps},{gam},100000\n"
                aggrfile.write(s)
aggrfile.close()

In [85]:
results = pd.read_csv(f"finals/{finalfile}")
results

,model,graph,n,p,pr,eps,gam,max_it
0,rewiring,er,250,0.1,0.0,0.2,0.0,100000
1,rewiring,er,250,0.1,0.0,0.2,0.4,100000
2,rewiring,er,250,0.1,0.0,0.2,0.8,100000
3,rewiring,er,250,0.1,0.0,0.2,1.2,100000
4,rewiring,er,250,0.1,0.0,0.2,1.6,100000
5,rewiring,er,250,0.1,0.0,0.3,0.0,100000
6,rewiring,er,250,0.1,0.0,0.3,0.4,100000
7,rewiring,er,250,0.1,0.0,0.3,0.8,100000
8,rewiring,er,250,0.1,0.0,0.3,1.2,100000
9,rewiring,er,250,0.1,0.0,0.3,1.6,100000


In [86]:
def nclusters(data, threshold):
    data = [float(el) for el in data]
    data = sorted(data)
    start = data[0]
    max_val = start + threshold
    c = (start, max_val)
    cluster = dict()
    for i in data:
        if i <= max_val:
            if c in cluster.keys():
                cluster[c] += 1
            else:
                cluster[c] = 1
        else:
            max_val = i + threshold
            c = (i, max_val)
            cluster[c] = 1
    #ora ho il dizionario con i cluster di una run
    C_num = len(data)**2
    C_den = 0
    for k in cluster.keys():
        C_den += cluster[k]*cluster[k]
    C = C_num / C_den
    return C

In [87]:
avg = []
std = []
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000'
    print(f'doing {name}')
    jsonfile = open(f'finals/final_iterations {name}.json')
    data = json.load(jsonfile)
    finalitarr = []
    for nr in data.keys():
        finalitarr.append(data[nr])
    finalitarr = np.array(finalitarr)
    avg.append(np.average(finalitarr))
    std.append(np.std(finalitarr))
results["avg_nit"] = avg
results["std_nit"] = std

doing rewiring er0.1 pr0.0 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0

In [88]:
avg01 = []
std01 = []
avg001=[]
std001=[]
avg00001=[]
std00001=[]
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000'
    print(f'doing {name}')
    jsonfile = open(f'finals/final_opinions {name}.json')
    data = json.load(jsonfile)
    ncarray01 = []
    ncarray001 = []
    ncarray00001 = []
    for nr in data.keys():
        finalops = list(data[nr])
        nc01 = nclusters(finalops, 0.1)
        nc001 = nclusters(finalops, 0.01)
        nc00001 = nclusters(finalops, 0.00001)
        ncarray01.append(nc01)
        ncarray001.append(nc001)
        ncarray00001.append(nc00001)
    ncarray01 = np.array(ncarray01)
    ncarray001 = np.array(ncarray001)
    ncarray00001 = np.array(ncarray00001)
    avg01.append(np.average(ncarray01))
    std01.append(np.std(ncarray01))
    avg001.append(np.average(ncarray001))
    std001.append(np.std(ncarray001))
    avg00001.append(np.average(ncarray00001))
    std00001.append(np.std(ncarray00001))
results["avg_ncluster_01"] = avg01
results["std_ncluster_01"] = std01
results["avg_ncluster_001"] = avg001
results["std_ncluster_001"] = std001
results["avg_ncluster_00001"] = avg00001
results["std_ncluster_00001"] = std00001

doing rewiring er0.1 pr0.0 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0

In [89]:
results.to_csv(f"finals/{finalfile}")

In [90]:
results = pd.read_csv(f"finals/{finalfile}", index_col=[0])
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,avg_ncluster_01,std_ncluster_01,avg_ncluster_001,std_ncluster_001,avg_ncluster_00001,std_ncluster_00001
0,rewiring,er,250,0.1,0.0,0.2,0.0,100000,146.5,85.597021,2.170596,0.476778,2.170636,0.476767,2.202778,0.512769
1,rewiring,er,250,0.1,0.0,0.2,0.4,100000,91.1,32.809907,2.413687,0.525845,2.413696,0.525832,2.542178,0.619739
2,rewiring,er,250,0.1,0.0,0.2,0.8,100000,591.7,172.078500,2.228691,0.386793,2.229085,0.386658,2.364724,0.382978
3,rewiring,er,250,0.1,0.0,0.2,1.2,100000,21772.0,6743.142843,2.579699,0.445250,2.645225,0.471204,2.742667,0.555480
4,rewiring,er,250,0.1,0.0,0.2,1.6,100000,82898.9,28598.886309,7.854710,0.402084,32.456240,5.061050,64.245012,10.079154


In [91]:
averageops = []
averagestdops = []
averagemedianops = []

for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000'
    print(f'doing {name}')
    jsonfile = open(f'finals/final_opinions {name}.json')
    data = json.load(jsonfile)
    avgarray = []
    stdarray = []
    medarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        avg = average_opinions(finalops)
        med = median_opinions(finalops)
        std = std_opinions(finalops)
        avgarray.append(avg)
        medarray.append(med)
        stdarray.append(std)
    avgarray = np.array(avgarray)
    medarray = np.array(medarray)
    stdarray = np.array(stdarray)
    averageops.append(np.average(avgarray))
    averagestdops.append(np.average(stdarray))
    averagemedianops.append(np.average(medarray))
results["avg_mean_opinion"] = averageops
results["avg_std_opinions"] = averagestdops
results["avg_median_opinion"] = averagemedianops

doing rewiring er0.1 pr0.0 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0

In [92]:
results.to_csv(f"finals/{finalfile}")

In [93]:
results = pd.read_csv(f"finals/{finalfile}", index_col=[0])
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,avg_ncluster_01,std_ncluster_01,avg_ncluster_001,std_ncluster_001,avg_ncluster_00001,std_ncluster_00001,avg_mean_opinion,avg_std_opinions,avg_median_opinion
0,rewiring,er,250,0.1,0.0,0.2,0.0,100000,146.5,85.597021,2.170596,0.476778,2.170636,0.476767,2.202778,0.512769,0.493026,0.235189,0.443834
1,rewiring,er,250,0.1,0.0,0.2,0.4,100000,91.1,32.809907,2.413687,0.525845,2.413696,0.525832,2.542178,0.619739,0.503692,0.240937,0.455000
2,rewiring,er,250,0.1,0.0,0.2,0.8,100000,591.7,172.078500,2.228691,0.386793,2.229085,0.386658,2.364724,0.382978,0.506752,0.237929,0.496806
3,rewiring,er,250,0.1,0.0,0.2,1.2,100000,21772.0,6743.142843,2.579699,0.445250,2.645225,0.471204,2.742667,0.555480,0.500606,0.249831,0.546255
4,rewiring,er,250,0.1,0.0,0.2,1.6,100000,82898.9,28598.886309,7.854710,0.402084,32.456240,5.061050,64.245012,10.079154,0.492880,0.277877,0.486494


In [94]:
values_avg = []
values_std = []

for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000'
    print(f'doing {name}')
    jsonfile = open(f'finals/final_opinions {name}.json')
    data = json.load(jsonfile)
    entrarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        entr = entropy(finalops, 100, 11)
        entrarray.append(entr)
    entrarray = np.array(entrarray)
    values_avg.append(np.average(entrarray))
    values_std.append(np.std(entrarray))

results["10B_avg_entr"] = values_avg
results["10B_std_entr"] = values_std

results['10B_avg_norm_entr'] = results['10B_avg_entr'].apply(lambda x: x/maxentr)

doing rewiring er0.1 pr0.0 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0

In [95]:
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,...,avg_ncluster_001,std_ncluster_001,avg_ncluster_00001,std_ncluster_00001,avg_mean_opinion,avg_std_opinions,avg_median_opinion,10B_avg_entr,10B_std_entr,10B_avg_norm_entr
0,rewiring,er,250,0.1,0.0,0.2,0.0,100000,146.5,85.597021,...,2.170636,0.476767,2.202778,0.512769,0.493026,0.235189,0.443834,0.789040,0.192599,0.343686
1,rewiring,er,250,0.1,0.0,0.2,0.4,100000,91.1,32.809907,...,2.413696,0.525832,2.542178,0.619739,0.503692,0.240937,0.455000,0.886833,0.201170,0.386282
2,rewiring,er,250,0.1,0.0,0.2,0.8,100000,591.7,172.078500,...,2.229085,0.386658,2.364724,0.382978,0.506752,0.237929,0.496806,0.830325,0.156175,0.361669
3,rewiring,er,250,0.1,0.0,0.2,1.2,100000,21772.0,6743.142843,...,2.645225,0.471204,2.742667,0.555480,0.500606,0.249831,0.546255,1.008177,0.179297,0.439137
4,rewiring,er,250,0.1,0.0,0.2,1.6,100000,82898.9,28598.886309,...,32.456240,5.061050,64.245012,10.079154,0.492880,0.277877,0.486494,2.220175,0.038018,0.967053


In [96]:
results.to_csv("finals/{}".format(finalfile))
results = pd.read_csv("finals/{}".format(finalfile), index_col=[0])

In [97]:
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,...,avg_ncluster_001,std_ncluster_001,avg_ncluster_00001,std_ncluster_00001,avg_mean_opinion,avg_std_opinions,avg_median_opinion,10B_avg_entr,10B_std_entr,10B_avg_norm_entr
0,rewiring,er,250,0.1,0.0,0.2,0.0,100000,146.5,85.597021,...,2.170636,0.476767,2.202778,0.512769,0.493026,0.235189,0.443834,0.789040,0.192599,0.343686
1,rewiring,er,250,0.1,0.0,0.2,0.4,100000,91.1,32.809907,...,2.413696,0.525832,2.542178,0.619739,0.503692,0.240937,0.455000,0.886833,0.201170,0.386282
2,rewiring,er,250,0.1,0.0,0.2,0.8,100000,591.7,172.078500,...,2.229085,0.386658,2.364724,0.382978,0.506752,0.237929,0.496806,0.830325,0.156175,0.361669
3,rewiring,er,250,0.1,0.0,0.2,1.2,100000,21772.0,6743.142843,...,2.645225,0.471204,2.742667,0.555480,0.500606,0.249831,0.546255,1.008177,0.179297,0.439137
4,rewiring,er,250,0.1,0.0,0.2,1.6,100000,82898.9,28598.886309,...,32.456240,5.061050,64.245012,10.079154,0.492880,0.277877,0.486494,2.220175,0.038018,0.967053


Unused functions

# def nclusters_meanshift(data, bw):
#     ops = {i: data[i] for i in range(len(data))}
#     sorted_ops = sorted(ops.items(), key = lambda kv:(kv[1], kv[0]))
#     A=np.array([el[1] for el in sorted_ops]).reshape(-1,1)
#     clustering = MeanShift(bandwidth=bw).fit(A)
#     lbls = clustering.labels_
#     labels = np.arange(len(sorted_ops))
#     for i in range(len(labels)):
#         cl = lbls[i]
#         labels[sorted_ops[i][0]]=cl
#     cluster_participation_dict = {}
#     for l in labels:
#         if l not in cluster_participation_dict:
#             cluster_participation_dict[l] = 1
#         else:
#             cluster_participation_dict[l] += 1
#     #computing effective number of clusters using function explained in the paper
#     C_num = 0
#     C_den = 0
#     for k in cluster_participation_dict:
#         C_num += cluster_participation_dict[k]
#         C_den += ((cluster_participation_dict[k])**2)
#     C_num = (C_num**2)
#     C = C_num/C_den
#     return C